In [1]:
import torch

#如果.requires_grad设置为True，Tensor将开始追踪在其上的所有操作
x = torch.ones(2, 2, requires_grad=True)
print(x)
print(x.grad_fn)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
None


In [2]:
#对原来的tensor做运算
y = x + 2
print(y)
print(y.grad_fn)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


由于x是直接创建的，所以没有`grad_fn`，而y是通过一个加法创建的，所以有一个为`<AddBackward>`的`grad_fn`

In [3]:
#更复杂的计算
z = y * y * 3
out = z.mean()
print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


也可以用`.requires_grad_()`来in-place地改变`requires_grad`属性

In [4]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)

a.requires_grad_(True)
print(a.requires_grad)

b = (a * a).sum()
print(b.grad_fn)

False
True


因为`out`是一个标量，所以调用`backward()`时不需要指定求导变量

In [5]:
out.backward() # 等价于 out.backward(torch.tensor(1.))
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


注意，grad在反向传播过程中是累加的，这意味着每一次运行反向传播，梯度都会累加之前的梯度，所以在反向传播之前将梯度清零

此外，如果我们想要修改`tensor`的数值，但是又不希望被`autograd`记录（即不会影响反向传播），那么我么可以对`tensor.data`进行操作。

In [6]:
x = torch.ones(1,requires_grad=True)

print(x.data) # 还是一个tensor
print(x.data.requires_grad) # 但是已经是独立于计算图之外

y = 2 * x
x.data *= 100 # 只改变了值，不会记录在计算图，所以不会影响梯度传播

y.backward()
print(x) # 更改data的值也会影响tensor的值
print(x.grad)


tensor([1.])
False
tensor([100.], requires_grad=True)
tensor([2.])
